In [1]:
import requests
import pandas as pd
import numpy as np
import cssselect
import lxml.html
import json
import lxml.objectify
from copy import copy

In [2]:

api_key = "42a8ecd3-5e1a-45a6-afc5-cf18846dc2c8"



In [3]:
bowie_gigs_url = "http://api.setlist.fm/rest/0.1/artist/5441c29d-3602-4898-b1a1-b77fa23b8e50/setlists.xml"

In [4]:
root = lxml.objectify.parse(bowie_gigs_url)
rooted = root.getroot()

OSError: Error reading file 'http://api.setlist.fm/rest/0.1/artist/5441c29d-3602-4898-b1a1-b77fa23b8e50/setlists.xml': failed to load external entity "http://api.setlist.fm/rest/0.1/artist/5441c29d-3602-4898-b1a1-b77fa23b8e50/setlists.xml"

In [16]:
rooted.attrib['total']

'1187'

In [17]:
pages = np.arange(1, 61)

In [18]:
gigs = []
data = ""
for i, p in enumerate(pages):
        #print i
    
        xml_url = bowie_gigs_url+"?p="+str(p)
        try:
            xml = lxml.objectify.parse(xml_url)
            data = xml
            root = xml.getroot()
            gigs.append(root)
        except:
            print "failed: %s" % (xml_url)
print len(gigs)

60


In [19]:
roots = xml.getroot()
print lxml.objectify.dump(root)

setlists = None [ObjectifiedElement]
  * itemsPerPage = '20'
  * page = '60'
  * total = '1187'
    setlist = None [ObjectifiedElement]
      * eventDate = '19-05-1968'
      * id = '2bc52c46'
      * lastUpdated = '2015-08-17T15:28:57.000+0000'
      * versionId = '5beb6fd0'
        artist = None [ObjectifiedElement]
          * disambiguation = ''
          * mbid = '5441c29d-3602-4898-b1a1-b77fa23b8e50'
          * name = 'David Bowie'
          * sortName = 'Bowie, David'
          * tmid = '734626'
            url = 'http://www.setlist.fm/setlists/david-bowie-6bd6ee66.html' [StringElement]
        venue = None [ObjectifiedElement]
          * id = '23d67c6f'
          * name = 'Middle Earth'
            city = None [ObjectifiedElement]
              * id = '2643743'
              * name = 'London'
              * state = 'England'
              * stateCode = 'ENG'
                coords = u'' [StringElement]
                  * lat = '51.5084152563931'
                  * long = '

In [29]:

def extract_data(page, data):
    songs_list = []
    for s in data.setlist:
        rec ={}
        rec['page'] = page
        rec['url'] = s.find('url')[0]
        #print rec['url']
        rec['event_date'] = s.attrib['eventDate']
        rec['tour'] = s.get('tour')
        #grab artist name
        try:
            rec['artist'] = s.find("artist").attrib['name']
        except:
            rec['artist'] = np.NaN
        #grab venue
        try:
            rec['venue'] = s.find("venue").attrib['name']
        except:
            rec['venue'] = np.NaN
        try:
            rec['lat'] = s.find("venue")['city']['coords'].attrib["lat"]
            rec['long']  = s.find("venue")['city']['coords'].attrib["long"]
        except:
            rec['lat'] = np.NaN
            rec['long']  = np.NaN

        try:
            rec['city'] = s.find("venue")['city'].attrib['name']
        except:
            rec['city'] = np.NaN
        try:
            rec['country'] = s.find("venue")['city']['country'].attrib["name"]
        except:
            rec['country'] = np.NaN

        try:
            set_list = s.find("sets")
            
            for sets in set_list:
                #print lxml.html.tostring(sets['set'])
                for num, song in enumerate(sets['set']):
                    
                    #print "len(songs) %s" %( len(sets['set']))
                    for son in song['song']:
                        #print lxml.html.tostring(son)
                        try:
                            rec['song'] = son.attrib['name']
                            
                        except:
                            rec['song'] = np.NaN
                            
                        
                        
                        try:
                            rec['cover'] = son['cover'].attrib['name']
                            

                        except:
                            
                            rec['cover'] = np.NaN
                        
                        
                        try:
                            rec['info'] = son['info']


                        except:
                            
                            rec['info'] = np.NaN
                        
                        sump = copy(rec)
                        
                        sump['song_title'] = sump['song'] 
                        
                        songs_list.append(sump)
                        
                        
                        

        except:
            print rec['url']
            print "wipeout"
            #song = {}
            #rec['song'] = np.NaN
            #song = rec
            #songs_list.append(song)
    return pd.DataFrame(songs_list)
ss = [extract_data(page, g)for page, g in enumerate(gigs)]
ss   

http://www.setlist.fm/setlist/david-bowie/1990/point-theatre-dublin-ireland-43f4c34f.html
wipeout
http://www.setlist.fm/setlist/david-bowie/1987/kooyong-stadium-melbourne-australia-4bc10fae.html
wipeout
http://www.setlist.fm/setlist/david-bowie/1987/kooyong-stadium-melbourne-australia-5bc10fac.html
wipeout
http://www.setlist.fm/setlist/david-bowie/1987/kooyong-stadium-melbourne-australia-43c10fa3.html
wipeout
http://www.setlist.fm/setlist/david-bowie/1987/sydney-entertainment-centre-sydney-australia-43c10fa7.html
wipeout
http://www.setlist.fm/setlist/david-bowie/1987/brisbane-entertainment-centre-brisbane-australia-13d8e9c9.html
wipeout
http://www.setlist.fm/setlist/david-bowie/1987/reunion-arena-dallas-tx-53c10f99.html
wipeout
http://www.setlist.fm/setlist/david-bowie/1987/rupp-arena-lexington-ky-5bc10f90.html
wipeout
http://www.setlist.fm/setlist/david-bowie/1987/marcus-amphitheater-milwaukee-wi-43c10f97.html
wipeout
http://www.setlist.fm/setlist/david-bowie/1987/marcus-amphitheater-

[          artist          city         country                   cover  \
 0    David Bowie      New York   United States           Johnny Mathis   
 1    David Bowie      New York   United States                     NaN   
 2    David Bowie      New York   United States                     NaN   
 3    David Bowie      New York   United States                     NaN   
 4    David Bowie      New York   United States             Arcade Fire   
 5    David Bowie      New York   United States                     NaN   
 6    David Bowie      Scheeßel         Germany                     NaN   
 7    David Bowie      Scheeßel         Germany                     NaN   
 8    David Bowie      Scheeßel         Germany                     NaN   
 9    David Bowie      Scheeßel         Germany                Iggy Pop   
 10   David Bowie      Scheeßel         Germany                     NaN   
 11   David Bowie      Scheeßel         Germany                  Pixies   
 12   David Bowie      Sc

In [25]:
len(ss)

60

In [22]:
songs_list[:15]

NameError: name 'songs_list' is not defined

In [30]:
    
songs = pd.concat(ss)
songs.shape


 

(19744, 14)

In [31]:
songs.head()

,artist,city,country,cover,event_date,info,lat,long,page,song,song_title,tour,url,venue
0,David Bowie,New York,United States,Johnny Mathis,09-11-2006,NaN,40.7142691,-74.0059729,0,Wild Is the Wind,Wild Is the Wind,None,[[[http://www.setlist.fm/setlist/david-bowie/2...,Hammerstein Ballroom
1,David Bowie,New York,United States,NaN,09-11-2006,NaN,40.7142691,-74.0059729,0,Fantastic Voyage,Fantastic Voyage,None,[[[http://www.setlist.fm/setlist/david-bowie/2...,Hammerstein Ballroom
2,David Bowie,New York,United States,NaN,09-11-2006,NaN,40.7142691,-74.0059729,0,Changes,Changes,None,[[[http://www.setlist.fm/setlist/david-bowie/2...,Hammerstein Ballroom
3,David Bowie,New York,United States,NaN,08-09-2005,[[[solo on piano]]],40.7142691,-74.0059729,0,Life on Mars?,Life on Mars?,None,[[[http://www.setlist.fm/setlist/david-bowie/2...,Radio City Music Hall
4,David Bowie,New York,United States,Arcade Fire,08-09-2005,NaN,40.7142691,-74.0059729,0,Wake Up,Wake Up,None,[[[http://www.setlist.fm/setlist/david-bowie/2...,Radio City Music Hall


In [33]:
songs.song.value_counts()

Fame                                                                                             628
The Jean Genie                                                                                   614
Rebel Rebel                                                                                      559
"Heroes"                                                                                         544
Suffragette City                                                                                 461
China Girl                                                                                       435
Stay                                                                                             411
Ziggy Stardust                                                                                   409
Fashion                                                                                          405
Life on Mars?                                                                              

In [34]:
songs[songs.song=="Wild Is the Wind"]

,artist,city,country,cover,event_date,info,lat,long,page,song,song_title,tour,url,venue
0,David Bowie,New York,United States,Johnny Mathis,09-11-2006,NaN,40.7142691,-74.0059729,0,Wild Is the Wind,Wild Is the Wind,None,[[[http://www.setlist.fm/setlist/david-bowie/2...,Hammerstein Ballroom
86,David Bowie,New York,United States,Johnny Mathis,24-07-2000,NaN,40.7142691,-74.0059729,8,Wild Is the Wind,Wild Is the Wind,None,[[[http://www.setlist.fm/setlist/david-bowie/2...,Studio 54
88,David Bowie,London,United Kingdom,Johnny Mathis,27-06-2000,NaN,51.5084152563931,-0.125532746315002,8,Wild Is the Wind,Wild Is the Wind,The Hours... Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,BBC Radio Theatre
110,David Bowie,Pilton,United Kingdom,Johnny Mathis,25-06-2000,NaN,51.0869868802263,-4.0620231628418,8,Wild Is the Wind,Wild Is the Wind,The Hours... Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,Worthy Farm
144,David Bowie,New York,United States,Johnny Mathis,19-06-2000,NaN,40.7142691,-74.0059729,8,Wild Is the Wind,Wild Is the Wind,The Hours... Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,Roseland Ballroom
167,David Bowie,New York,United States,Johnny Mathis,16-06-2000,NaN,40.7142691,-74.0059729,8,Wild Is the Wind,Wild Is the Wind,The Hours... Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,Roseland Ballroom
51,David Bowie,Munich,Germany,Johnny Mathis,21-05-1983,NaN,48.1374325498109,11.5754914283752,35,Wild Is the Wind,Wild Is the Wind,Serious Moonlight Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Olympiahalle
76,David Bowie,Frankfurt,Germany,Johnny Mathis,20-05-1983,NaN,50.1166667,8.6833333,35,Wild Is the Wind,Wild Is the Wind,Serious Moonlight Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Festhalle
103,David Bowie,Brussels,Belgium,Johnny Mathis,19-05-1983,NaN,50.8465974826927,4.35277462005615,35,Wild Is the Wind,Wild Is the Wind,Serious Moonlight Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Forest National / Vorst Nationaal
134,David Bowie,Brussels,Belgium,Johnny Mathis,18-05-1983,NaN,50.8465974826927,4.35277462005615,35,Wild Is the Wind,Wild Is the Wind,Serious Moonlight Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Forest National / Vorst Nationaal


In [36]:
songs[songs.country == "United Kingdom"].city.unique()

array(['Newport', 'Glasgow', 'London', 'Birmingham', 'Manchester',
       'Pilton', 'Leeds', 'Nottingham', 'Newcastle', 'Liverpool',
       'Stratford-upon-Avon', 'Belfast', 'Sheffield', 'Aberdeen',
       'Cardiff', 'Exeter', 'Milton Keynes', 'Edinburgh', 'Sunderland',
       'Stafford', 'Oxford', 'Salisbury', 'Chatham', 'Southampton',
       'Hanley', 'Guildford', 'Brighton', 'Preston', 'Bristol',
       'Aylesbury', 'Dunstable', 'Slough', 'Hertford',
       'Kingston upon Thames', 'Sutton Coldfield', 'Eltham', 'Falmouth'], dtype=object)

In [37]:
songs[(songs.country == "United Kingdom")&
      (songs.city.isin(['Hanley', "Stafford"]))]


,artist,city,country,cover,event_date,info,lat,long,page,song,song_title,tour,url,venue
152,David Bowie,Stafford,United Kingdom,NaN,26-06-1978,NaN,52.8,-2.1,36,Warszawa,Warszawa,Isolar II Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Bingley Hall
153,David Bowie,Stafford,United Kingdom,NaN,26-06-1978,NaN,52.8,-2.1,36,"""Heroes""","""Heroes""",Isolar II Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Bingley Hall
154,David Bowie,Stafford,United Kingdom,NaN,26-06-1978,NaN,52.8,-2.1,36,What in the World,What in the World,Isolar II Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Bingley Hall
155,David Bowie,Stafford,United Kingdom,NaN,26-06-1978,NaN,52.8,-2.1,36,Be My Wife,Be My Wife,Isolar II Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Bingley Hall
156,David Bowie,Stafford,United Kingdom,NaN,26-06-1978,NaN,52.8,-2.1,36,The Jean Genie,The Jean Genie,Isolar II Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Bingley Hall
157,David Bowie,Stafford,United Kingdom,NaN,26-06-1978,NaN,52.8,-2.1,36,Blackout,Blackout,Isolar II Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Bingley Hall
158,David Bowie,Stafford,United Kingdom,NaN,26-06-1978,NaN,52.8,-2.1,36,Sense of Doubt,Sense of Doubt,Isolar II Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Bingley Hall
159,David Bowie,Stafford,United Kingdom,NaN,26-06-1978,NaN,52.8,-2.1,36,Speed of Life,Speed of Life,Isolar II Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Bingley Hall
160,David Bowie,Stafford,United Kingdom,NaN,26-06-1978,NaN,52.8,-2.1,36,Breaking Glass,Breaking Glass,Isolar II Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Bingley Hall
161,David Bowie,Stafford,United Kingdom,NaN,26-06-1978,NaN,52.8,-2.1,36,Fame,Fame,Isolar II Tour,[[[http://www.setlist.fm/setlist/david-bowie/1...,Bingley Hall


In [42]:
songs[songs.cover.notnull()]['cover'].value_counts()

Iggy Pop                                 576
The Velvet Underground                   486
Queen                                    293
Pixies                                   156
The McCoys                               121
Adrian Belew                              97
Kurt Weill                                87
Laurie Anderson                           71
Scott Walker                              68
Jacques Brel                              64
The Rolling Stones                        50
Neil Young                                48
Tin Machine                               37
The Modern Lovers                         37
Them                                      32
Eddie Floyd                               23
The Beatles                               19
The Who                                   17
Chuck Berry                               15
Iggy and The Stooges                      14
Johnny Mathis                             13
The Flairs                                12
George Har

In [ ]:
songs.ix[12288]['url']

In [38]:
songs[songs['info'].notnull()]

,artist,city,country,cover,event_date,info,lat,long,page,song,song_title,tour,url,venue
3,David Bowie,New York,United States,NaN,08-09-2005,[[[solo on piano]]],40.7142691,-74.0059729,0,Life on Mars?,Life on Mars?,None,[[[http://www.setlist.fm/setlist/david-bowie/2...,Radio City Music Hall
35,David Bowie,Prague,Czech Republic,NaN,23-06-2004,[[[David is injured and has to leave the stage]]],50.0878367932108,14.4241322001241,0,Reality,Reality,A Reality Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,T-Mobile Arena
36,David Bowie,Prague,Czech Republic,NaN,23-06-2004,[[[band only]]],50.0878367932108,14.4241322001241,0,A New Career in a New Town,A New Career in a New Town,A Reality Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,T-Mobile Arena
37,David Bowie,Prague,Czech Republic,NaN,23-06-2004,[[[Catherine Russell on vocals]]],50.0878367932108,14.4241322001241,0,Be My Wife,Be My Wife,A Reality Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,T-Mobile Arena
40,David Bowie,Prague,Czech Republic,NaN,23-06-2004,[[[interrupted and restarted after interruptio...,50.0878367932108,14.4241322001241,0,Station to Station,Station to Station,A Reality Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,T-Mobile Arena
42,David Bowie,Prague,Czech Republic,NaN,23-06-2004,"[[[interrupted, end of show]]]",50.0878367932108,14.4241322001241,0,Changes,Changes,A Reality Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,T-Mobile Arena
162,David Bowie,Amsterdam,Netherlands,The Velvet Underground,11-06-2004,[[[Cut short]]],52.373,4.9,0,White Light/White Heat,White Light/White Heat,A Reality tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,ArenA
315,David Bowie,Atlantic City,United States,NaN,29-05-2004,[[[NIN V1 Mix]]],39.3642834,-74.4229266,0,I'm Afraid of Americans,I'm Afraid of Americans,A Reality Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,Borgata Events Center
37,David Bowie,Pittsburgh,United States,NaN,17-05-2004,[[[band introductions followed]]],40.4406248,-79.9958864,1,Heathen (The Rays),Heathen (The Rays),A Reality Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,Benedum Center
74,David Bowie,Fairfax,United States,NaN,16-05-2004,[[[NIN V1 Mix]]],38.8462236,-77.3063733,1,I'm Afraid of Americans,I'm Afraid of Americans,A Reality Tour,[[[http://www.setlist.fm/setlist/david-bowie/2...,Patriot Center


In [39]:
set_lists = []
def set_lists_extract(s):
    try:
        print "http://api.setlist.fm/rest/0.1/search/setlists?venueId="+str(s)
        root = lxml.objectify.parse("http://api.setlist.fm/rest/0.1/search/setlists?venueId="+str(s)).getroot()
        total_items =  int(root.attrib['total'])
        print "total pages: %s" % (str(total_items))
        items_per_page = int(root.attrib['itemsPerPage'])
        print "items per page: %s" % (str(items_per_page))
        page_no = int(root.attrib['page'])
        print "page no.: %s" % (str(page_no))
        pages_total = int(round(total_items *1.0 / items_per_page, 0))
        print "page total.: %s" % (str(pages_total))
        for r in root['setlist']:
            set_lists.append(r.getchildren()[-1])
        #Extra steps if page_total > 1
        extra_pages = np.arange(2, pages_total+1)
        for i in extra_pages:
            print "loop page: %s" %(str(i))
            root = lxml.objectify.parse("http://api.setlist.fm/rest/0.1/search/setlists?venueId="+str(s)+"&p="+str(i)).getroot()
            for r in root['setlist']:
                set_lists.append(r.getchildren()[-1])
    except:
        "http://api.setlist.fm/rest/0.1/search/setlists?venueId="+str(s)+ " failed!"
venues['id'].map(set_lists_extract)    
#set_lists_extract("1bd66db0")

NameError: name 'venues' is not defined

In [ ]:
set_lists[0]

In [ ]:
def extract_set_list_string(s):
    str_txt = s.text
    return str_txt.split("-")[-1].replace(".html", "")
set_id_numbers = [extract_set_list_string(s) for s in set_lists]


In [ ]:
set_id_numbers

In [ ]:
set_deets_str = "http://api.setlist.fm/rest/0.1/setlist/"
set_deets_str_2 = ".json"

In [ ]:
def other_attributes(s):
    print s
    url = set_deets_str + s + set_deets_str_2
    data = pd.read_json(url)
    return data
deets = [other_attributes(s) for s in set_id_numbers]

In [ ]:
gigs = pd.concat(deets, axis=1).T
gigs = gigs.reset_index().drop("index", axis=1)

In [ ]:
gigs['Artist'] = gigs.artist.map(lambda x: x["@name"])

gigs['sort_name'] = gigs.artist.map(lambda x: x["@sortName"])

gigs['venue'] = gigs.venue.map(lambda x: x['@name'])

In [ ]:

gigs.artist.ix[0]

In [ ]:
for s in gigs.ix[0].sets['set']:
    print s['song'][s]["@name"]
    

In [ ]:
gigs['set'] = gigs.sets.map(lambda x: x['set'] if x else np.NaN)

In [ ]:
gigs.ix[0]['sets']

In [ ]:
gigs['date'] = pd.to_datetime(gigs["@eventDate"], dayfirst=True)

In [ ]:
def process_set(d):
    print d
    if str(d) != "nan":
        print len(d)
    
        if len(d) >= 1 and isinstance(d, list):
            print "LIST"
            for song in d:
                print "LISTING"
                if isinstance(song['song'], list):
                    print "LISTY"
                    for s in song['song']:
                        print s["@name"]
                        if "cover" in s.keys():
                            print "COVER SONG BY"
                            print s['cover']["@name"]
                        if "@encore" in song['song'][0].keys():
                            print "ENCORE"
                if isinstance(song['song'], dict):
                    print song['song']['@name']
                    print "CHOB"
                    if "@encore" in song.keys():
                        print "ENCORE"
        else:
            print "DICT"
            print d['song']
            if len(d['song']) >1:
                for s in d['song']:
                        print s["@name"]
                        if "cover" in s.keys():
                            print "COVER SONG BY"
                            print s['cover']["@name"]
                        if "@encore" in s.keys():
                            print "ENCORE"
            else:
                print d['song']["@name"]
                if "@encore" in d['song'].keys():
                        print "ENCORE"
    else:
        print ""
gigs['set'].map(process_set)
    

In [ ]:
gigs['year'] = gigs['date'].dt.year
gigs['year'].value_counts()

In [ ]:
gigs.ix[0]['venue']

In [ ]:
gigs.venue.unique()

In [ ]:
gigs.Artist.unique()

In [ ]:
gigs[gigs.Artist.str.match("Stranglers")]